In [1]:
import numpy as np
import math
import random
import os
import pybullet as p
import pybullet_data
import gym
from gym import error, spaces, utils
from gym.utils import seeding
import matplotlib.pyplot as plt
import time

from math import sqrt
from skimage.feature import blob_dog, blob_log, blob_doh

In [2]:
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Input, Dropout, concatenate
from keras.layers import Concatenate,Conv2D,BatchNormalization,MaxPooling2D
from keras.optimizers import Adam
from keras import initializers

C:\Users\SERVER\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\SERVER\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\SERVER\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\SERVER\Anaconda3\lib\site-packages\tensorflow\python\framew

In [3]:
from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import  SequentialMemory
from rl.processors import Processor
from rl.callbacks import Callback

In [4]:
class BikeEnv(gym.Env):
    
    def __init__(self):
        p.connect(p.GUI)
        p.setRealTimeSimulation(1)
        p.resetDebugVisualizerCamera(cameraDistance=10, cameraYaw=0, cameraPitch=-40, cameraTargetPosition=[0.55,-0.35,0.2])
        self.action_space = spaces.Discrete(6)
        self.observation_space = spaces.Dict({
            'sensor' : spaces.Box(np.array([-1000]*10), np.array([1000]*10)),
            'image'  : spaces.Box(low=0, high=1, shape=(64, 64))
        }) 
        
        self.timestep = 1./240.
        self.stp = 0
        
    def step(self, action):
        #print("i=%d" % self.stp)
        self.stp+=1
        #print("time = " , time.time()-self.time)
     
        if (action == 0):
            self.speed = self.speed + 1
        if (action == 1):
            self.speed = self.speed - 1 
        if (action == 2):
            self.speed = self.speed  
        if (action == 3):
            self.steer = self.steer - 1 
        if (action == 4):
            self.steer = self.steer + 1
        if (action == 5):
            self.steer = self.steer 
            
              
        self.applyAction([self.speed,self.steer])

        #p.setRealTimeSimulation(1)
        time.sleep(0.5)
        #p.setRealTimeSimulation(0)
        
        state = p.getLinkState(self.pid,0)[0]
        if state[2] <= 0.5 or  state[2] >= 2 or abs(self.speed)>2 or abs(self.steer)>4:
            reward = -100
            done = True
        else :
            #reward = math.sqrt((self.origin[0]-state[0])**2+(self.origin[1]-state[1])**2)
            reward = state[0] - self.origin[0]
            #reward = 1
            done = False
        self.origin = state 
        
        velocity = p.getBaseVelocity(self.pid)
        img = self.getImage()
        observation = {'sensor':list(self.getObservation()) + list(velocity[0])+list(velocity[1]),
                       'image': img}
        
        info = {'x':'','y':'','z':''}
        #print("Step: ",self.stp)
        #xx = time.time()
        #print("Time: ",xx-self.tttt)
        #self.tttt = xx
        #print("Action: ",action)
        #print("Reward: ",reward)
        #self.stp +=1
        return observation, reward, done, info
            
    def applyAction(self, motorCommands):
        targetVelocity = motorCommands[0] * self.speedMultiplier
        #print("targetVelocity")
        #print(targetVelocity)
        steeringAngle = motorCommands[1] * self.steeringMultiplier
        #print("steeringAngle")
        #print(steeringAngle)


        for motor in self.motorizedwheels:
            p.setJointMotorControl2(self.pid,
                                    motor,
                                    p.VELOCITY_CONTROL,
                                    targetVelocity=targetVelocity,
                                    force=self.maxForce)
        for steer in self.steeringLinks:
            p.setJointMotorControl2(self.pid,
                                    steer,
                                    p.POSITION_CONTROL,
                                    targetPosition=steeringAngle)

    def reset(self):
        #print("Reset")
        #print("setp:",self.stp)
        self.stp = 0

        p.resetSimulation()

        urdfRootPath = pybullet_data.getDataPath()
        planeUid = p.loadURDF(os.path.join(urdfRootPath,"plane.urdf"), basePosition=[0,0,0])
        
        
        for i in range(np.random.randint(1,10)):
            p.loadURDF(os.path.join(urdfRootPath, "sphere2.urdf"),basePosition=[
                np.random.randint(5,15),
                np.random.randint(-2,2),
               0.5
           ])
     
        self.pid = p.loadURDF(os.path.join(urdfRootPath, "bicycle/bike.urdf"),basePosition=[0,0,1])     
           
        
        self.origin = p.getLinkState(self.pid,0)[0]
        p.setGravity(0,0,-10)
        for wheel in range(p.getNumJoints(self.pid)):
            p.setJointMotorControl2(self.pid,
                                    wheel,
                                    p.VELOCITY_CONTROL,
                                    targetVelocity=0,
                                    force=0)

        self.steeringLinks = [0]
        self.maxForce = 20
        self.nMotors = 2
        self.motorizedwheels = [1, 2]
        self.speedMultiplier = 10.
        self.steeringMultiplier = 0.5
        
        self.speed = 0 
        self.steer = 0

        velocity = p.getBaseVelocity(self.pid)
        img = self.getImage()
        observation = {'sensor': list(self.getObservation()) + list(velocity[0])+list(velocity[1]),
                       'image': img}
        

        p.configureDebugVisualizer(p.COV_ENABLE_RENDERING,1)
        
        return observation
        
    
    def getImage(self):
        img = env.render()
        #img = cv2.resize(img, (64,64), interpolation = cv2.INTER_NEAREST )
        #img = img[:,:,2]
        #img =img.reshape((64,64))
        #img = np.asarray(img, dtype='float32')
        #img /= 255.0
        return img
        
    def getObservationDimension(self):
        return len(self.getObservation())
    
    def getObservation(self):
        observation = []
        pos, orn = p.getBasePositionAndOrientation(self.pid)

        #observation.extend(list(pos))
        observation.extend(list(orn))
        return observation
        
    def render(self, mode='rgb_array'):
        pos, orn = p.getBasePositionAndOrientation(self.pid)
      
        view_matrix = p.computeViewMatrixFromYawPitchRoll(cameraTargetPosition=[pos[0]+11.3,pos[1],pos[2]],
                                                            distance=10,
                                                            yaw=-90 ,
                                                            pitch=0,
                                                            roll=0,
                                                            upAxisIndex=2)
        proj_matrix = p.computeProjectionMatrixFOV(fov=60,
                                                     aspect=float(960) /720,
                                                     nearVal=0.1,
                                                     farVal=100.0)
        (_, _, px, _, s) = p.getCameraImage(width=64,
                                              height=64,
                                              viewMatrix=view_matrix,
                                              projectionMatrix=proj_matrix,
                                              renderer=p.ER_BULLET_HARDWARE_OPENGL)

        #rgb_array = np.array(px, dtype=np.uint8)
        #rgb_array = np.reshape(rgb_array, (64,64, 4))

        #rgb_array = rgb_array[:, :, :3]
        
        #return rgb_array
        for i in range(64):
            for j in range(64):
                if s[i,j] <= 0 :
                    s[i,j] = 0
                else:
                    s[i,j] = 1
        image = s 
        image = np.zeros((64,64))
        for i in range(64):
            for j in range(64):
                if s[i,j] > 0 : image[i,j]=1

        blobs_doh = blob_doh(image, max_sigma=30, threshold=.01)



        for blob in blobs_doh:
            print('blob',blob)
          
        return s
    
    def close(self):
        p.disconnect()

In [5]:
env = BikeEnv()
np.random.seed(123)
env.seed(123)

C:\Users\SERVER\Anaconda3\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [6]:
nb_actions = env.action_space.n

In [7]:
def create_mlp():
    inputs = Input(shape=env.observation_space['sensor'].shape)
    x = Dense(16, activation='relu')(inputs)
    x = Dense(16, activation='relu')(x)
    x = Dense(16, activation='relu')(x)
    x = Dense(8,  activation='relu')(x)
    #x = Dense(nb_actions, activation='linear')(x)
    model = Model(inputs, x)
    return model

In [8]:
init = initializers.RandomNormal(stddev=0.01)

In [9]:
def create_cnn():
    inputShape = (64, 64,1)
    inputs = Input(shape=inputShape,name='image')
    x = Conv2D(32, (8, 78), padding="same", strides=(4,4), kernel_initializer=init)(inputs)
    x = Activation("relu")(x)

    
    x = Conv2D(64, (4, 4), padding="same" ,strides=(2,2), kernel_initializer=init)(x)
    x = Activation("relu")(x)

    x = Conv2D(64, (3, 3), padding="same" ,strides=(1,1), kernel_initializer=init)(x)
    x = Activation("relu")(x)
  
    x = Flatten()(x)
    x = Dense(512,  kernel_initializer=init)(x)
    x = Activation("relu")(x)
   
    #x = Dense(6)(x)
    #x = Activation("linear")(x)
 
    model = Model(inputs, x)
    return model

In [10]:
mlp = create_mlp()
cnn = create_cnn()
combinedInput = concatenate([mlp.output, cnn.output])
x = Dense(8, activation="relu")(combinedInput)
x = Dropout(0.2)(x)
x = Dense(nb_actions, activation="relu")(x)
model = Model(inputs=[mlp.input, cnn.input], outputs=x)

In [11]:
class CustomCalback(Callback ):
    def on_action_begin(self, action, logs={}):
        p.setRealTimeSimulation(1)

    def on_action_end(self, action, logs={}):
        p.setRealTimeSimulation(0)

In [12]:
class CustomProcessor(Processor):

    def process_state_batch(self, batch):
        batch = np.squeeze(batch, axis=1)
        b_shape = batch.shape[0]
        sensor = [batch[i][0] for i in range(b_shape)]
        images = [batch[i][1] for i in range(b_shape)]
        return [sensor,images]

        return batch
    def process_observation(self, observation):
        
        observation = observation['sensor'],observation['image'].reshape((64,64,1))
        return observation

In [13]:
memory = SequentialMemory(limit=50000, window_length=1)
policy = BoltzmannQPolicy()
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=10,
               target_model_update=1e-2, policy=policy)
dqn.processor = CustomProcessor()
dqn.compile(Adam(lr=1e-6), metrics=['mae'])

In [14]:
dqn.fit(env, nb_steps=250000, visualize=False, verbose=1,callbacks  = [CustomCalback()])

Training for 250000 steps ...
blob [33.         35.          4.22222222]
Interval 1 (0 steps performed)

blob [33.         36.          4.22222222]
    3/10000 [..............................] - ETA: 1:50:02 - reward: -33.3398

C:\Users\SERVER\Anaconda3\lib\site-packages\skimage\feature\blob.py:125: RuntimeWarning: invalid value encountered in double_scalars
  r1 = blob1[-1] / blob2[-1]
C:\Users\SERVER\Anaconda3\lib\site-packages\skimage\feature\blob.py:126: RuntimeWarning: divide by zero encountered in true_divide
  pos1 = blob1[:ndim] / (max_sigma * root_ndim)
C:\Users\SERVER\Anaconda3\lib\site-packages\skimage\feature\blob.py:127: RuntimeWarning: divide by zero encountered in true_divide
  pos2 = blob2[:ndim] / (max_sigma * root_ndim)
C:\Users\SERVER\Anaconda3\lib\site-packages\skimage\feature\blob.py:129: RuntimeWarning: invalid value encountered in subtract
  d = np.sqrt(np.sum((pos2 - pos1)**2))


blob [36.         48.          7.44444444]
blob [33.         27.          4.22222222]
blob [37.         52.          7.44444444]
blob [34.         41.          4.22222222]
blob [33.         28.          4.22222222]
    4/10000 [..............................] - ETA: 2:14:29 - reward: -24.8582blob [33.         28.          4.22222222]
blob [35.         56.         13.88888889]
    5/10000 [..............................] - ETA: 2:06:49 - reward: -19.6764blob [ 0.         53.         17.11111111]
blob [ 0. 14.  1.]
blob [21.         40.         10.66666667]
blob [ 2. 16.  1.]
blob [ 4. 20.  1.]
blob [ 5. 23.  1.]
blob [ 6. 25.  1.]
blob [ 7. 27.  1.]
blob [ 8. 29.  1.]
blob [ 9. 30.  1.]
blob [ 3. 19.  1.]
    6/10000 [..............................] - ETA: 2:03:51 - reward: -33.0636blob [36.         31.         10.66666667]
blob [38.         31.         10.66666667]
    7/10000 [..............................] - ETA: 2:19:11 - reward: -28.2592

C:\Users\SERVER\Anaconda3\lib\site-packages\skimage\feature\blob.py:127: RuntimeWarning: invalid value encountered in true_divide
  pos2 = blob2[:ndim] / (max_sigma * root_ndim)
C:\Users\SERVER\Anaconda3\lib\site-packages\skimage\feature\blob.py:126: RuntimeWarning: invalid value encountered in true_divide
  pos1 = blob1[:ndim] / (max_sigma * root_ndim)


blob [46.         42.         23.55555556]
blob [ 3.          4.         23.55555556]
    9/10000 [..............................] - ETA: 2:31:14 - reward: -32.8874blob [38.         53.         10.66666667]
blob [36.         26.         13.88888889]
blob [28.         43.          7.44444444]
blob [47.         40.         10.66666667]
blob [41.         39.         20.33333333]
   11/10000 [..............................] - ETA: 2:38:11 - reward: -27.1467blob [30.         41.         10.66666667]


...

blob [37.         21.          7.44444444]
blob [33.         35.          4.22222222]
blob [37.         22.          7.44444444]
blob [33.         35.          4.22222222]
   13/10000 [..............................] - ETA: 3:21:44 - reward: -30.6629blob [36.         25.          7.44444444]
blob [33.         36.          4.22222222]
   15/10000 [..............................] - ETA: 3:17:18 - reward: -33.2412blob [34.         33.          7.44444444]
blob [34.         33.          7.44444444]
   16/10000 [..............................] - ETA: 3:24:54 - reward: -31.2004blob [33.         37.          4.22222222]
blob [32. 27.  1.]
blob [31. 30.  1.]
   17/10000 [..............................] - ETA: 3:21:32 - reward: -29.4729blob [30.         37.          4.22222222]
blob [30. 29.  1.]
blob [28. 30.  1.]
   18/10000 [..............................] - ETA: 3:18:32 - reward: -33.3910blob [38.         29.         13.88888889]


C:\Users\SERVER\Anaconda3\lib\site-packages\skimage\feature\blob.py:127: RuntimeWarning: invalid value encountered in true_divide
  pos2 = blob2[:ndim] / (max_sigma * root_ndim)


blob [40.         24.         20.33333333]
   20/10000 [..............................] - ETA: 3:30:18 - reward: -30.0805

C:\Users\SERVER\Anaconda3\lib\site-packages\skimage\feature\blob.py:126: RuntimeWarning: invalid value encountered in true_divide
  pos1 = blob1[:ndim] / (max_sigma * root_ndim)


blob [29.         37.         10.66666667]
blob [20.          0.         23.55555556]
   21/10000 [..............................] - ETA: 3:29:54 - reward: -33.4100blob [33.         36.          4.22222222]
blob [33.         27.          4.22222222]
blob [33.         36.          4.22222222]
blob [33.         27.          4.22222222]
   22/10000 [..............................] - ETA: 3:37:41 - reward: -31.8915blob [33.         38.          4.22222222]
blob [33.         28.          4.22222222]
   23/10000 [..............................] - ETA: 3:34:51 - reward: -30.5047blob [29.         42.          4.22222222]
blob [30.         31.          4.22222222]
blob [25.         36.          4.22222222]
   24/10000 [..............................] - ETA: 3:32:15 - reward: -33.4003blob [36.         39.          7.44444444]
blob [41.         30.          7.44444444]
blob [32. 22.  1.]
blob [31. 23.  1.]
blob [36.         39.          7.44444444]
blob [41.         31.          7.44444444]
blob 

In [15]:
#dqn.save_weights('d:\\RL-DQN-16')

In [16]:
dqn.load_weights('d:\\RL-DQN-16')

OSError: Unable to open file (unable to open file: name = 'd:\RL-DQN-16', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
# get a ride
all_obs = []
env.reset()
for i in range(100):
    action = env.action_space.sample()
    observation, reward, done, info = env.step(action)
    all_obs.append(observation)
    if done :
        env.reset()

In [ ]:
# visualize observations
for i in range(100):
    plt.imshow(all_obs[i]['image'])
    plt.show()

In [ ]:
# Visualize Filters
for layer in model.layers:
    # check for convolutional layer
    if 'conv' not in layer.name:
        continue
    # get filter weights
    filters, biases = layer.get_weights()
    print(layer.name, filters.shape)

In [ ]:
n_filters, ix = 6, 1
for i in range(n_filters):
    # get the filter
    f = filters[:, :, :, i]
    # plot each channel separately
    for j in range(1):
        # specify subplot and turn of axis
        ax = plt.subplot(n_filters, 3, ix)
        ax.set_xticks([])
        ax.set_yticks([])
        # plot filter channel in grayscale
        plt.imshow(f[:, :, j], cmap='gray')
        ix += 1
# show the figure
plt.show()

In [ ]:
# get model oputput
for i in range(100):
    sensor = np.asarray(all_obs[i]['sensor']).reshape((1,10))
    image = all_obs[i]['image'].reshape((1,64,64,1))
    print(model.predict([sensor,image]))

In [ ]:
# visualize feature map
image = all_obs[66]['image']
sensor = np.asarray(all_obs[3]['sensor']).reshape((1,10))
plt.imshow(image , cmap='gray')

In [ ]:
m = Model(inputs=model.inputs, outputs=model.layers[6].output)
m.summary()
image = image.reshape((1,64,64,1))
feature_maps = m.predict([sensor , image])
for i in range(32):
    plt.imshow(feature_maps[0, :, :, i], cmap='gray')
    plt.show()